In [37]:
DATE = "0201_ncc_abn_normal"
DATA_TO_TEST = 'normal' # 'abn1' or 'abn2' or 'normal'
MODEL_EPOCH = '_47ep' ###### 아래 스냅샷 파일과 일치하는지 반드시 확인!!!!!!! ######
SCORE_VALUE = 0.1
PRINT_BENIGN_DETECTION = False

import os

result_save_base_path = '/home/huray/workspace/keras-retinanet/results/'
result_save_path = os.path.join(result_save_base_path, DATE, DATA_TO_TEST + MODEL_EPOCH + '_' + str(SCORE_VALUE))
if not os.path.exists(result_save_path):
    os.makedirs(result_save_path)

In [38]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import keras
import keras.preprocessing.image
from keras_retinanet.models.resnet import custom_objects
# from keras_retinanet.preprocessing.coco import CocoGenerator
from keras_retinanet.preprocessing.csv_generator import CSVGenerator

import matplotlib.pyplot as plt
import cv2

import numpy as np
import time

import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
keras.backend.tensorflow_backend.set_session(get_session())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load RetinaNet model

In [39]:
model = keras.models.load_model('snapshots/0201_ncc_abn_normal/resnet101_csv_47.h5', custom_objects=custom_objects)

#print(model.summary())

/home/huray/venvs/ml/lib/python3.5/site-packages/keras/models.py:252: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


## Initialize data generators

In [40]:
# create image data generator object
val_image_data_generator = keras.preprocessing.image.ImageDataGenerator()

# create a generator for testing data
val_generator = CSVGenerator(
    '/home/huray/data/NCC/img_retinanet/data_' + DATA_TO_TEST + '.csv',
    '/home/huray/data/NCC/img_retinanet/class_3.csv',
    val_image_data_generator,
    batch_size=1
)

f = open('/home/huray/data/NCC/img_retinanet/data_' + DATA_TO_TEST + '.csv', 'r')
num_of_test_files = len(f.readlines())
print(num_of_test_files)
f.close()

353


## Run detection

In [41]:
no_detection_counter = 0

for index in range(num_of_test_files):
#     print(index)
    flag = False
    
    # load image
    image, image_path = val_generator.load_image(index, is_testing=True)

    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = val_generator.preprocess_image(image)
    image, scale = val_generator.resize_image(image)
    annotations = val_generator.load_annotations(index)
    index += 1

    # process image
    start = time.time()
    _, _, detections = model.predict_on_batch(np.expand_dims(image, axis=0))
#     print("processing time: ", time.time() - start)

    # compute predicted labels and scores
    predicted_labels = np.argmax(detections[0, :, 4:], axis=1)
    scores = detections[0, np.arange(detections.shape[1]), 4 + predicted_labels]

    # correct for image scale
    detections[0, :, :4] /= scale
    

    # visualize detections
    for idx, (label, score) in enumerate(zip(predicted_labels, scores)):
        if score < SCORE_VALUE:
            continue
#         else:
#             print(score)
#             flag = True

        b = detections[0, idx, :4].astype(int)
        if len(b) == 0:
            print('NO DETECTION! ', index)
            no_detection_counter += 1
            
        caption = "{} {:.3f}".format(val_generator.label_to_name(label), score)
        
        
        ######### BENIGN 출력 여부 설정 #########
        if PRINT_BENIGN_DETECTION is False:
            if 'B' in caption:
                continue
            
            
        cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 0), 5)
        cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 4)
        
        cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (0, 255, 0), 5)
#         print(b)
        

    # visualize annotations
    for annotation in annotations:
        label = int(annotation[4])
        b = annotation[:4].astype(int)
        cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (50, 50, 255), 2)
        caption = "{}".format(val_generator.label_to_name(label))
        cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 0), 5)
        cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 4)
        

    cv2.imwrite(os.path.join(result_save_path, image_path.split('/')[-1].split('.')[0]+'_'+str(index)+'.jpg'), draw)
        
#     plt.figure(figsize=(12, 12))
#     plt.axis('off')
#     plt.imshow(draw)
#     plt.show()

    
# print(no_detection_counter)

/home/huray/data/NCC/img_retinanet/normal/normal-00025-20004_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00051-30001_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00032-40004_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00002-30003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00044-20003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00020-20004_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00092-30003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00068-40002_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00002-30002_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00079-40003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00025-20002_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00013-0004_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00064-40002_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00004-40002_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00005-40004_0.

/home/huray/data/NCC/img_retinanet/normal/normal-00041-40001_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00026-30002_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00071-40002_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00023-0003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00046-30002_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00019-20001_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00066-30001_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00054-30003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00074-20001_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00063-20002_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00012-30001_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00042-40004_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00047-30002_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00012-30003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00073-40002_0.

/home/huray/data/NCC/img_retinanet/normal/normal-00018-30004_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00023-0004_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00056-30003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00029-40004_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00006-30003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00061-30001_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00060-20003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00087-20003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00075-10002_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00062-30001_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00080-20002_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00086-20001_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00024-20003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00007-30002_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00028-30003_0.

In [42]:
print("DONE!" + MODEL_EPOCH)

DONE!_47ep
